# **보이스피싱 탐지 프로그램 개발 코드**

In [ ]:
#드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**데이터 수집 및 전처리**

### unzip mp3

In [ ]:
%cd /content/drive/MyDrive/voicevishing_detection_proj-main
!unzip -qq "/content/drive/MyDrive/voicevishing_detection_proj-main/mp3file.Zip"

/content/drive/MyDrive/voicevishing_detection_proj-main
replace voice_file187.mp3? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### install pydub

In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment
import os

### remove corrupted or bad data

In [ ]:
directory = "/content/drive/MyDrive/voicevishing_detection_proj-main/mp3file/"

for filename in os.listdir(directory):
    if filename.endswith('.mp3'):
        try:
            AudioSegment.from_file(os.path.join(directory, filename), format="mp3")
        except Exception as e:
            print(f"Error reading {filename}: {e}")
            os.remove(os.path.join(directory, filename))
            print(f"{filename} removed.")

In [ ]:
directory = "/content/drive/MyDrive/voicevishing_detection_proj-main/mp3file/"
export_directory = "/content/drive/MyDrive/voicevishing_detection_proj-main/mp3file_10dbup/"

for filename in os.listdir(directory):
  sound = AudioSegment.from_file(directory+filename, format="mp3")

  increased_dB = sound + 10
  newname = export_directory+filename[:-4]+"_10dbup.mp3"
  increased_dB.export(newname, format="mp3")
  print(newname+"converted")

/content/drive/MyDrive/voicevishing_detection_proj-main/mp3file_10dbup/voice_file187_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_detection_proj-main/mp3file_10dbup/voice_file188_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_detection_proj-main/mp3file_10dbup/voice_file189_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_detection_proj-main/mp3file_10dbup/voice_file191_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_detection_proj-main/mp3file_10dbup/voice_file190_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_detection_proj-main/mp3file_10dbup/voice_file192_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_detection_proj-main/mp3file_10dbup/voice_file193_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_detection_proj-main/mp3file_10dbup/voice_file194_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_detection_proj-main/mp3file_10dbup/voice_file195_10dbup.mp3converted
/content/drive/MyDrive/voicevishing_detection_proj-main

### convert to wav

In [ ]:
directory = "/content/drive/MyDrive/voicevishing_detection_proj-main/mp3file_10dbup/"
export_directory = "/content/drive/MyDrive/voicevishing_detection_proj-main/wavfile/"

i = int(0)
for filename in os.listdir(directory):
  i += 1
  dst = export_directory + "/audio%d.wav"%i
  audSeg = AudioSegment.from_mp3(directory+filename)
  audSeg.export(dst, format="wav")                      # 텍스트 파일로 변환하기 위해 mp3파일을 wav파일로 변환함


### convert to text file

In [ ]:
! pip install SpeechRecognition

In [ ]:
# wav 파일을 불러와 text파일로 변환
import os
import speech_recognition as sr
import wave

r = sr.Recognizer()

input_dir = "/content/drive/MyDrive/voicevishing_detection_proj-main/wavfile"
output_dir = "/content/drive/MyDrive/voicevishing_detection_proj-main/vishing_text"

segment_length = 30 #sec

wav_files = [f for f in os.listdir(input_dir) if f.endswith('.wav')]

for wav_file in wav_files:
    print("Processing file: ", wav_file)

    # get the duration of the WAV file in seconds
    with wave.open(os.path.join(input_dir, wav_file), 'r') as wave_file:
        audio_duration = wave_file.getnframes() / float(wave_file.getframerate())

    # calculate the number of segments
    num_segments = int(audio_duration / segment_length) + 1

    # loop over each segment
    for i in range(num_segments):
        print("Processing segment: ", i+1)

        start_time = i * segment_length
        end_time = min((i+1) * segment_length, audio_duration)

        # load the audio segment
        with sr.AudioFile(os.path.join(input_dir, wav_file)) as source:
            audio_segment = r.record(source, offset=start_time, duration=end_time-start_time)

        try:
            text = r.recognize_google(audio_segment, language='ko')
            print("Text: ", text)

            text_file = os.path.splitext(wav_file)[0] + f"_segment_{i+1}.txt"
            with open(os.path.join(output_dir, text_file), "w") as f:
                f.write(text)
                print("Text file saved!")
        except sr.UnknownValueError:
            print("Could not understand audio")
        except sr.RequestError as e:
            print("Could not request results; {0}".format(e))


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Text file saved!
Processing segment:  50
Could not understand audio
Processing segment:  51
Text:  작년 여름에 작년 내려 작년 여름에게 걸어 주고 어느 지점인지 기억이 나세요 본인 계좌에 사업자 계좌는 전혀 없으시죠네 없으시고 주말에 보고서 확인해 드릴게요
Text file saved!
Processing segment:  52
Text:  여보세요 경남은행에서 처음에 개설하신게 언제쯤 될 거라 하셨습니까
Text file saved!
Processing segment:  53
Text:  제일 마지막에 계산할게요 마지막이에요 거의 비슷한 시기인 거 같은데 잘 모르겠으니까 3년 전에 주세요
Text file saved!
Processing segment:  54
Text:  친구 와 가지고 잠깐 이야기하는 거예요 아파트에서 예속 복제된 신용카드 1200점 발견을 했습니다
Text file saved!
Processing segment:  55
Text:  본인 신용카드 본인이 직접 개설하신게 몇 가지가 있으시죠 좀 찾아볼래 차가워요 많고 없으시고 하니까 잘 모르겠는데 그 부분이 문제가 되는 거예요 일곱 개에서 8가지 사용하시는 분이 있었어요 자주 사용하시는 신용카드만 사용하시기 때문에 신용카드를 분실하신지도 모르시고 계시네요
Text file saved!
Processing segment:  56
Text:  신용카드 개설하신 본인이 개설하신 거 다 소지하고 계신 거 아니세요 생겼다 이거 본인이 신용카드 그랬으면 들고 있으신데 몇 가지 가지고 계세요 다 정리하고
Text file saved!
Processing segment:  57
Text:  가격 약 없으시다고요 자 본인 지금 입력해 주신 이용자 아이디 오류가 나셨어요네 본인 뭐라고 기대하셨던 이용자 아이디가 오류가 낮았어요 잠깐만요 예
Text file saved

### text processing

In [37]:
import os
dir_path = '/content/drive/MyDrive/voicevishing_detection_proj-main/vishing_text'
min_length = 5

for filename in os.listdir(dir_path):
  if filename.endswith('.txt'):
    file_path = os.path.join(dir_path, filename)

    with open(file_path, 'r', encoding='utf-8') as file_:
      file_contents = file_.read()
      if len(file_contents) < min_length:       # 5글자 이하 삭제
        os.remove(file_path)
      elif not file_contents.endswith('\t1'):
        with open(file_path, 'a', encoding='utf-8') as output_file:
          output_file.write('\t1')            #Tab 해서 뒤에 1 붙이기

In [38]:
dir_path = '/content/drive/MyDrive/voicevishing_detection_proj-main/vishing_text'
output_file = '/content/drive/MyDrive/voicevishing_detection_proj-main/pos.txt'

with open(output_file, 'w', encoding='utf-8') as out_file:
    for filename in os.listdir(dir_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(dir_path, filename)

            with open(file_path, 'r') as in_file:
                file_contents = in_file.read()
                out_file.write(file_contents.strip() + '\n')

In [39]:
dir = '/content/drive/MyDrive/voicevishing_detection_proj-main/'
with open(dir+'pos.txt', 'r', encoding='utf-8') as file:
  pos_cnt = 0
  for line in file:
    pos_cnt += 1
print(f'pos text contains {pos_cnt} line')

with open(dir+'neg_text.txt', 'r', encoding='utf-8') as file:
  neg_cnt = 0
  for line in file:
    neg_cnt += 1
print(f'neg text contains {neg_cnt} line')

pos text contains 3196 line
neg text contains 100000 line


neg_text와 줄단위로 랜덤으로 병합

In [40]:
import random

with open(dir+'neg_text.txt', 'r', encoding='utf-8') as file:
  lines = file.readlines()

num_lines_to_del = abs(neg_cnt-pos_cnt)
print(f'will delte {num_lines_to_del} lines')

lines_to_del = random.sample(range(neg_cnt), num_lines_to_del)

with open(dir+'neg_text_pruned.txt', 'w', encoding='utf-8') as pruned_file:
  for i, line in enumerate(lines):
    if i in lines_to_del:
      continue
    pruned_file.write(line)

will delte 96804 lines


In [41]:
dir = '/content/drive/MyDrive/voicevishing_detection_proj-main/'

with open(dir+'pos.txt', 'r', encoding='utf-8') as f1, open(dir+'neg_text_pruned.txt', 'r', encoding='utf-8') as f2:
    lines = f1.readlines() + f2.readlines()

random.shuffle(lines)

with open(dir+'voice_phishing.txt', 'w', encoding='utf-8') as f:
    for i, line in enumerate(lines):
        if i % 2 == 0:
            f.write(line.strip() + '\n')
        else:
            f.write(line.strip() + '\n')


#**딥러닝 모델 구축**

### import

In [42]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Flatten, LSTM, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np

### load dataset and tokenize and split

In [43]:
def load_dataset(file_path):
  texts = []
  labels = []
  with open(file_path, 'r') as f:
    for line in f:
      parts = line.strip().split('\t')
      texts.append(parts[0])
      labels.append(int(parts[1]))
  return texts, labels

In [44]:
dir = '/content/drive/MyDrive/voicevishing_detection_proj-main/'
texts, labels = load_dataset(dir+'voice_phishing.txt')

labels = np.array(labels)
num_classes = len(set(labels))
labels = np.eye(num_classes)[labels]

### tokenize

In [86]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_sequence_length = 50
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [87]:
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1
indices = np.arange(len(texts))
np.random.shuffle(indices)
train_indices = indices[:int(train_ratio * len(texts))]
val_indices = indices[int(train_ratio * len(texts)):int((train_ratio+val_ratio) * len(texts))]
test_indices = indices[int((train_ratio+val_ratio) * len(texts)):]

train_data = padded_sequences[train_indices]
train_labels = labels[train_indices]
val_data = padded_sequences[val_indices]
val_labels = labels[val_indices]
test_data = padded_sequences[test_indices]
test_labels = labels[test_indices]

### make model

In [88]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=50, input_shape=(max_sequence_length,)))
model.add(Bidirectional(LSTM(units=128, dropout=0.2)))
model.add(Dropout(0.5))
model.add(Dense(units=num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 50, 50)            1547100   
                                                                 
 bidirectional_3 (Bidirecti  (None, 256)               183296    
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 514       
                                                                 
Total params: 1730910 (6.60 MB)
Trainable params: 1730910 (6.60 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### train

In [79]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [80]:
model.fit(
    train_data,
    train_labels,
    epochs=10,
    batch_size=32,
    validation_data=(val_data, val_labels),
    callbacks=[early_stopping]
)

Epoch 1/10
160/160 [==============================] - 62s 343ms/step - loss: 0.2906 - accuracy: 0.8750 - val_loss: 0.0729 - val_accuracy: 0.9671
Epoch 2/10
160/160 [==============================] - 54s 338ms/step - loss: 0.1392 - accuracy: 0.9542 - val_loss: 0.0721 - val_accuracy: 0.9750
Epoch 3/10
160/160 [==============================] - 60s 374ms/step - loss: 0.0298 - accuracy: 0.9898 - val_loss: 0.0598 - val_accuracy: 0.9828
Epoch 4/10
160/160 [==============================] - 53s 333ms/step - loss: 0.0103 - accuracy: 0.9971 - val_loss: 0.0600 - val_accuracy: 0.9781
Epoch 5/10
160/160 [==============================] - 52s 328ms/step - loss: 0.0035 - accuracy: 0.9990 - val_loss: 0.0868 - val_accuracy: 0.9624
Epoch 6/10
160/160 [==============================] - 54s 336ms/step - loss: 0.0078 - accuracy: 0.9990 - val_loss: 0.0759 - val_accuracy: 0.9734


### evaluate

In [81]:
test_loss, test_acc = model.evaluate(test_data, test_labels, batch_size=32)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

20/20 [==============================] - 2s 74ms/step - loss: 0.0782 - accuracy: 0.9656
Test loss: 0.07818392664194107
Test accuracy: 0.965624988079071


### save

In [82]:
dir = '/content/drive/MyDrive/voicevishing_audio/'
model.save(dir+'voice_phishing_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### TEST

In [52]:
!pip install konlpy

In [53]:
from konlpy.tag import Okt
from string import punctuation
import re
from keras.models import load_model
import numpy as np

In [83]:
okt = Okt()

def preprocess_text(text):
    okt = Okt()
    morphemes = okt.morphs(text)
    return ' '.join(morphemes)

In [84]:
model = load_model(dir+'voice_phishing_model.h5')

def predict_phishing(text):
    preprocessed_text = preprocess_text(text)
    sequence = tokenizer.texts_to_sequences([preprocessed_text])
    padded_sequence = pad_sequences(sequence, maxlen=50)
    predictions = model.predict(padded_sequence)
    phishing_probability = predictions[0][1]
    is_phishing = int(phishing_probability > 0.5)
    return is_phishing, phishing_probability

In [91]:
text=input()
predict, prob = predict_phishing(text)

if predict > 0.5:
  print(f"{prob:.3f} 확률로 보이스피싱")
else:
  print(f"{1-prob:.3f} 확률로 보이스피싱 아님")

안녕하세요 저희는 해당 프로그램을 개발한 밤양갱갱갱입니다.
1/1 [==============================] - 0s 35ms/step
0.508 확률로 보이스피싱 아님


#**모델 사용자화 - NAVER CLOVA API (CLOVA speech Recognition CSR)**

In [57]:
!pip install requests

In [58]:
%cd /content/drive/MyDrive/call_sample

/content/drive/MyDrive/call_sample


In [59]:
ls

 sample_voice.mp3   test2.mp3   test.mp3  '대검찰청 보이스피싱.mp3'


# 통화 녹음 파일 탐지

In [95]:
import requests
import json
import os

# 네이버 클라우드 플랫폼에서 발급받은 API 키
client_id = "ibmzidnlpk"
client_secret = "U541j23f4VCEFlEvJXdec0HrRLkpky2j48YMCUph"
url = "https://naveropenapi.apigw.ntruss.com/recog/v1/stt?lang=Kor"  # 한국어로 설정

# 사용자가 업로드한 통화 녹음 파일 받아옴
directory_sample_voice = "/content/drive/MyDrive/call_sample/"

filenames = []
for filename in os.listdir(directory_sample_voice):
  filenames.append(filename)

audio_file_path = directory_sample_voice + filenames[-1]     # 가장 최근에 업로드된 파일 불러와서 텍스트 변환
print(audio_file_path)

headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret,
    "Content-Type": "application/octet-stream"
}

with open(audio_file_path, 'rb') as audio_file:
    response = requests.post(url, headers=headers, data=audio_file)

if response.status_code == 200:
    print("Transcription successful!")
    print(json.loads(response.text))
else:
    print(f"Error: {response.status_code}")
    print(response.text)

/content/drive/MyDrive/call_sample/test2.mp3
Transcription successful!
{'text': '담당 하시는 분 성함이랑 방 번호는 어디있지 대검찰청 우리은행 계좌를 통해서 확인 미술 단기 테솔 자격증 이렇게 드시면 식은 해당 사건 관련해서 명의도용 피해자인데요 이제 보이나 보다 하고 사건에 대해서 지하철 1-8회가 신들이 중요한 것들은 이제 김민수 이번 다음 주 제주도의 대표적인 조사를 해서 녹취 자료로 법정 증거자료로 채택이 들어가실 거에요 본인 동의 하에 진행될 예정으로 시는분이 계속 그랬는데 그러면 제 번호와 이름은 어떤 경로로 일어나는 일이 필요하신것은 그런것까지 하나하나 필요하신 거예요 어 그러면'}


확률 계산 후 결과 출력

In [98]:
predict, prob = predict_phishing(response.text)

if predict > 0.5:
  print(f"{prob:.3f} 확률로 보이스피싱")
else:
  print(f"{1-prob:.3f} 확률로 보이스피싱 아님")

1/1 [==============================] - 0s 32ms/step
0.500 확률로 보이스피싱 아님


# 문자파일 탐지

In [68]:
%cd /content/drive/MyDrive/sms_sample

/content/drive/MyDrive/sms_sample


In [69]:
ls

sms1.gdoc  sms2  sms3  sms4


In [70]:
import os

# 사용자가 업로드한 문자 파일 받아옴
directory_sample_text = "/content/drive/MyDrive/sms_sample/"

filenames = []
for filename in os.listdir(directory_sample_text):
  filenames.append(filename)

sms_file_path = directory_sample_text + filenames[-1]     # 가장 최근에 업로드된 파일 경로 불러옴
print(sms_file_path)

/content/drive/MyDrive/sms_sample/sms4


In [99]:
sms_text = ""
with open(sms_file_path, "r", encoding="utf8") as inFile:      #파일 내용 읽기
  lines = inFile.readlines()
for line in lines:
  sms_text += line
print(sms_text)

sms = sms_text.replace("\n","")     # 줄바꿈 제거
print(sms)

제목없음
안녕하세요, 학사팀입니다.
내일 마이크로디그리 신청이 마감됩니다.
2~6학기 재학·휴학 중인 학부생 모두 신청할 수 있으니 여러분의 많은 관심 바랍니다.
상세 공지는 아래 링크를 확인해 주십시오.
제목없음안녕하세요, 학사팀입니다.내일 마이크로디그리 신청이 마감됩니다.2~6학기 재학·휴학 중인 학부생 모두 신청할 수 있으니 여러분의 많은 관심 바랍니다.상세 공지는 아래 링크를 확인해 주십시오.


In [100]:
predict, prob = predict_phishing(sms)

if predict > 0.5:
  print(f"{prob:.3f} 확률로 보이스피싱")
else:
  print(f"{1-prob:.3f} 확률로 보이스피싱 아님")

1/1 [==============================] - 0s 35ms/step
0.507 확률로 보이스피싱 아님


# **결과 문자 전송**

In [101]:
!pip install twilio

account_sid = "AC5f2de1b96a33716d4b4dcc66b7f6ce59"
auth_token = "d969a94f50686ca40cfa631e797445f3"
from_number = "+18126377377"
to_number = "+821064843176"

from twilio.rest import Client
client = Client(account_sid, auth_token)

if predict > 0.8:
  message = client.messages.create(
    to = to_number,
    from_ = from_number,
    body = f"위험합니다. 보이스피싱일 확률이 {prob:.3f}입니다."
  )
elif predict > 0.6:
  message = client.messages.create(
    to = to_number,
    from_ = from_number,
    body = f"경고입니다. 보이스핑일 확률이 {prob:.3f}입니다."
  )
elif predict > 0.5:
  message = client.messages.create(
    to = to_number,
    from_ = from_number,
    body = f"조심하세요. 보이스피싱일 확률이 {prob:.3f}입니다."
  )
else:
  message = client.messages.create(
    to = to_number,
    from_ = from_number,
    body = f"안심하세요. 보이스피싱이 아닐 확률이 {1-prob:.3f}입니다."
  )

print(f"Message SID: {message.sid}")

Message SID: SM7dfc10693f064a3c7092650b6ed19ef1
